In [1]:
#first job returns 

#import packages
from datetime import datetime
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
import re

In [2]:
# let's first make a function that will make the process of building a url easy.
def make_url(base_url , comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url, r)
        
    return url

# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    return(np.unique(x)) 

In [3]:
#returns a list of 10-k urls  (summary files)
def pull_10k_urls():

    start = time.time()
    print("Starting to Pull All 10K URLs ")

    list_10k = []

    year_list = [ '2011','2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
    for y in year_list: 
        
        try: 
            print("Pulling Year: "+y)
            #get all 2019 10ks into dataframe of company, date, balance sheet, income statement, cash flow statement

            # PULL DAILY INDEX FILINGS 
            base_url = r"https://www.sec.gov/Archives/edgar/daily-index"

            # The daily-index filings, require a year and content type (html, json, or xml).
            year_url = make_url(base_url, [y, 'index.json'])

            # Display the new Year URL
            # print('-'*100)
            print('Building the URL for Year: {}'.format(y))
            print("URL Link: " + year_url)

            # request the content for 2019, remember that a JSON strucutre will be sent back so we need to decode it.
            content = requests.get(year_url)
            decoded_content = content.json()

            # def get_10k():
            for item in decoded_content['directory']['item']:

                # get the name of the folder
                print('-'*100)
                print('Pulling url for Quarter: {}'.format(item['name']))

                # The daily-index filings, require a year, a quarter and a content type (html, json, or xml).
                qtr_url = make_url(base_url, [y, item['name'], 'index.json'])

                # print out the url.
                print("URL Link: " + qtr_url)

                # Request, the new url and again it will be a JSON structure.
                file_content = requests.get(qtr_url)
                sleep(0.2)
                decoded_content = file_content.json()

                print('-'*100)
                print('Pulling files')

                #get just the master file urls into a df 
                master_url_list = []
                for file in decoded_content['directory']['item']:
                    if "master" in file['name']:
                        file_url = make_url(base_url, [y, item['name'], file['name']]).replace(".gz","")

                        master_url_list.append(file_url)
                        #print("File URL Link: " + file_url)
                        #sleep(2)
                    else:
                        pass
                #print(master_url_df) 

                #traverse the master url files 
                for u in master_url_list: 
                    #get content of the file 
                    print("Requesting: " + u)
                    content = requests.get(u).content
                    sleep(0.2)  

                    try: 
                        data = content.decode("utf-8").split('  ')
                        # We need to remove the headers, so look for the end of the header and grab it's index
                        for index, item in enumerate(data):
                            if "ftp://ftp.sec.gov/edgar/" in item:
                                start_ind = index
                        data_format = data[start_ind + 1:]
                        #list to store master data info 
                        master_data = []
                        # now we need to break the data into sections, this way we can move to the final step of getting each row value.
                        for index, item in enumerate(data_format):

                            # if it's the first index, it won't be even so treat it differently
                            if index == 0:
                                clean_item_data = item.replace('\n','|').split('|')
                                clean_item_data = clean_item_data[8:]
                            else:
                                clean_item_data = item.replace('\n','|').split('|')

                            for index, row in enumerate(clean_item_data):

                                # when you find the text file.
                                if '.txt' in row:

                                    # grab the values that belong to that row. It's 4 values before and one after.
                                    mini_list = clean_item_data[(index - 4): index + 1]

                                    if len(mini_list) != 0:
                                        mini_list[4] = "https://www.sec.gov/Archives/" + mini_list[4]
                                        master_data.append(mini_list)
        #                                 print(mini_list)
                                #print(master_data)

                            master_data = pd.DataFrame.from_records(master_data, columns = ['cik', 'name', 'form','date', 'url'])
                           # print(master_data)
                            df_10k = master_data[master_data['form'] == "10-K"]
                            #print(len(df_10k))
                            if len(df_10k) > 0:
                                list_10k.append(df_10k)
                                print(df_10k)
                            #print(list_10k)
                    except Exception as ex: 
                        print("url failed: "+ u)
                        pass
        except Exception as ex:
            print ("something broke")

    #End Code
    end = time.time()
    print(end - start)
    print("Finished Pulling 10K URLs")
    df_10k = pd.concat(url_list_10k)
    df_10k = df_10k.drop_duplicates()
    
    #write 10k links to csv
    df_10k.to_csv("df_10k.csv")
    
    #read in 10k df 
    df_10k_extract = pd.read_csv("df_10k.csv")

    #iterate through 10k urls and return df of urls that lead to filing summary 
    print("Start Pulling Filing Summaries")
    start = time.time()
    summary_url_df = []

    base_url = r"https://www.sec.gov"

    for index, row in df_10k_extract.iterrows():
        #print(row)
        doc_url = row['url'].replace('-','').replace('.txt','/index.json')
        print("Requesting: "+doc_url)
        content = requests.get(doc_url).json()
        #rate limit of 10/second
        sleep(0.2)

        for file in content['directory']['item']:
            # Grab the filing summary and create a new url leading to the file so we can download it.
            if file['name'] == 'FilingSummary.xml':
                xml_summary = base_url + content['directory']['name'] + "/" + file['name']
                new_row = [row['cik'], row['name'], row['date'],  xml_summary]
                summary_url_df.append(new_row)

    summary_url_df = pd.DataFrame.from_records(summary_url_df, columns = ['cik', 'name', 'date', 'xml_summary'])
    #print(summary_url_df['xml_summary'][0])

    #write summary urls to a csv
    summary_url_df.to_csv("summary_urls.csv")
    
    #read in summary urls from csv 
    summary_url_extract = pd.read_csv("summary_urls.csv")

    #End Code
    end = time.time()
    print(end - start)
    print("Finished Pulling Filing Summaries")
    
    return(summary_url_extract)
    